In [ ]:
import obsidian
print(f'obsidian version: ' + obsidian.__version__)

import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

## Introduction

In this tutorial, we will examine usage of _obsidian_ for performing a cost-penalized optimization using a tailored objective function.

Often times, it is desireable to generate an objective function based on the input data `X`. Rather than build a model to a response calculated off-line, it is best to capture the analytical form where possible. The custom objective `Feature_Objective` simply allows the user to index the input variables and multiply them by coefficient(s) to generate a new objective function. In this example, we will create a  `Feature_Objective` based off of "Enzyme" loading, to simulate an optimization where product yield might be weighed against an expensive input.

The optimization problem then becomes multi-output: "Product" and "Penalized Enzyme." However, we can combine these further to create a single objective function using a `Scalarization`. In the simplest case, we may want to add the two objectives with equal weights, which would be the default behavior of `Scalarize_WeightedSum`.

In _obsidian_, we combine a sequence of objectives using `Objective_Sequence`. Thus, finally, the final objective function is `objective = Objective_Sequence([Feature_Objective, Scalarize_WeightedSum])` and single-output acquisition functions may be used to select optimal experiments.

## Set up parameter space and initialize a design

In [ ]:
from obsidian import Campaign, Target, ParamSpace, BayesianOptimizer
from obsidian.parameters import Param_Continuous

In [ ]:
params = [
    Param_Continuous('Temperature', -10, 30),
    Param_Continuous('Concentration', 10, 150),
    Param_Continuous('Enzyme', 0.01, 0.30),
    ]

X_space = ParamSpace(params)
target = Target('Product', aim='max')
campaign = Campaign(X_space, target, seed=0)
X0 = campaign.initialize(m_initial = 10, method = 'LHS')

X0

## Collect results (e.g. from a simulation)

In [ ]:
from obsidian.experiment import Simulator
from obsidian.experiment.benchmark import cornered_parab

simulator = Simulator(X_space, cornered_parab, name='Product', eps=0.05)
y0 = simulator.simulate(X0)
Z0 = pd.concat([X0, y0], axis=1)

campaign.add_data(Z0)
campaign.data.sample(5)

## Fit the optimizer and visualize results

In [ ]:
campaign.fit()

In [ ]:
from obsidian.plotting import factor_plot, optim_progress

In [ ]:
factor_plot(campaign.optimizer, feature_id=1)

## Optimize new experiment suggestions

In [ ]:
from obsidian.objectives import Objective_Sequence, Feature_Objective, Scalar_WeightedSum

Note: Objectives can be passed directly to an `Optimizer`, or set using `campaign.set_objective()` after which the `Campaign` will automatically use the objective durign `campaign.suggest()`. At any time, the objective can be re-set to a new objective, or deleted using `campaign.clear_objective()`.

In [ ]:
penalize_enz_loading = Feature_Objective(X_space, indices=[2], coeff=[-5])
add_objectives = Scalar_WeightedSum(1)

campaign.set_objective(objective=Objective_Sequence([penalize_enz_loading, add_objectives]))

In [ ]:
X_suggest, eval_suggest = campaign.suggest(m_batch = 3, optim_sequential = False)

In [ ]:
df_suggest = pd.concat([X_suggest, eval_suggest], axis=1)
df_suggest

Note: We can examine the output of various objectives within the sequence by passing them directly to `optimizer.evaluate`. Here, we can explicitly see the balance of Objective 1 (product response) and Objective 2 (cost penalty) before they are combined in the weighted sum.

In [ ]:
campaign.optimizer.evaluate(X_suggest, objective=penalize_enz_loading)

## Collect data at new suggestions

In [ ]:
y_iter1 = pd.DataFrame(simulator.simulate(X_suggest), columns = ['Product'])
Z_iter1 = pd.concat([X_suggest, y_iter1, eval_suggest], axis=1)
campaign.add_data(Z_iter1)
campaign.data.tail()

## Repeat as desired

In [ ]:
for iter in range(3):
    campaign.fit()
    X_suggest, eval_suggest = campaign.suggest(m_batch=3)
    y_iter = pd.DataFrame(simulator.simulate(X_suggest))
    Z_iter = pd.concat([X_suggest, y_iter, eval_suggest], axis=1)
    campaign.add_data(Z_iter)

Examine the optimization progress from the context of different elements of the compositve objective function

First, the final objective - a weighted sum of product yield and a cost-penalized input.

In [ ]:
optim_progress(campaign, color_feature_id = 'aq Value')

Next, we can specifically examine the context of the multi-output optimization minimizing (maximizing negative) cost (Objective 2) and maximizing product (Objective 1).

In [ ]:
campaign.set_objective(penalize_enz_loading)
optim_progress(campaign)


Finally, we can clear the objective entirely and just examine how this optimization performed from the lens of product alone.

In [ ]:
campaign.clear_objective()
optim_progress(campaign)